In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, RobustScaler,
    MaxAbsScaler, PowerTransformer,
    QuantileTransformer, Normalizer
)

class ScalingSystem:
    def __init__(self, df_initial: pd.DataFrame, df_scaled_path: str, X: list):
        self.df_initial = df_initial
        self.df_scaled_path = df_scaled_path
        self.X = X

        if os.path.exists(self.df_scaled_path):
            self.df_scaled = pd.read_csv(self.df_scaled_path)
        else:
            self.df_scaled = df_initial.copy()
            self.df_scaled.to_csv(self.df_scaled_path, index=False)

    def method(self, tool_name: str, config: dict):

        if tool_name == "StandardScaling":
            scaler = StandardScaler(**config)

        elif tool_name == "MinMaxScaler":
            scaler = MinMaxScaler(**config)

        elif tool_name == "RobustScaler":
            scaler = RobustScaler(**config)

        elif tool_name == "MaxAbsScaler":
            scaler = MaxAbsScaler(**config)

        elif tool_name == "PowerTransformer":
            scaler = PowerTransformer(**config)

        elif tool_name == "QuantileTransformer":
            scaler = QuantileTransformer(**config)

        elif tool_name == "UnitVectorScaler":
            scaler = Normalizer(**config)

        else:
            return "Invalid scaling method"

        scaled_values = scaler.fit_transform(self.df_scaled[self.X])
        scaled_df = pd.DataFrame(scaled_values, columns=self.X)

        self.df_scaled[self.X] = scaled_df
        self.df_scaled.to_csv(self.df_scaled_path, index=False)

        return self.df_scaled

In [ ]:
import os
import pandas as pd 
class Normalization:
    def __init__(self, df_scaled_path: str, df_final_path: str, X: list):
        self.df_scaled_path = df_scaled_path
        self.df_final_path = df_final_path
        self.X = X

        if os.path.exists(self.df_scaled_path):
            self.df_input = pd.read_csv(self.df_scaled_path)

        else:
            return "Scaled dataset not found"

        if os.path.exists(self.df_final_path):
            self.df_final = pd.read_csv(self.df_final_path)

        else:
            self.df_final = self.df_input.copy()
            self.df_final.to_csv(self.df_final_path, index=False)


    def method(self, tool_name: str, config: dict):
        if tool_name == "Normalizer":
            transformer = Normalizer(**config)
            values = transformer.fit_transform(self.df_final[self.X])

        elif tool_name == "BoxCox":
            transformer = PowerTransformer(method="box-cox", **config)
            values = transformer.fit_transform(self.df_final[self.X])

        elif tool_name == "YeoJohnson":
            transformer = PowerTransformer(method="yeo-johnson", **config)
            values = transformer.fit_transform(self.df_final[self.X])

        elif tool_name == "Clipping":
            lower = config.get("lower")
            upper = config.get("upper")
            if lower is None or upper is None:
                return "lower and upper required"

            values = np.clip(self.df_final[self.X], lower, upper)

        else:
            return "Invalid normalization method"

        transformed_df = pd.DataFrame(values, columns=self.X)
        self.df_final[self.X] = transformed_df
        self.df_final.to_csv(self.df_final_path, index=False)

        return self.df_final